# <center>Trabajo Práctico - Twitter API<center>
***


## Introducción
En éste trabajo presentamos una pequeña aplicación encargada de la recolección y presentación de información proporcionada por la **API** de **Twitter**. Ésta misma buscará aquellos tweets que nos resulten interesantes según la temáctica que hayamos elegido.


***

## Query
A la hora de elegir un contenido para la investigación nos tomamos el trabajo de seleccionar un temática concurrente para el desarrollo de una consulta firme y robusta.
Decidimos utilisar el tópico de cambio climático, y sus temas relacionados, para la construcción de la siguiente QUERY.

***QUERY = "cambio climático OR sequías OR calentamiento global OR economía circular OR espacios verde OR protección ambiental lang:es -is:retweet"***

Con ésta misma anduvimos recolecantdo una gran cantidad de tweets para el desarrollo de una estrucutura compuesta para el procesamiento, lectura y servicio de los datos almacenados.

***

## Recolección de Tweets

Con el siguiente código pudimos recolectar una cierta cantidad de Tweets. Modificamos el mismo, proporcionado por las presentaciones dadas en clase, para que sea más optimo para nuestra tarea. Incluimos una limpieza de las ***STREAM RULES***, esto mismo lo vimos importante debido a que, cada vez que realizabamos una nueva busqueda, estas mismas quedaban con la información de la busqueda previa. Como consecuencia, nos quedaba "información basura" para futuras consultas. También incluimos la hora en la que se inicia la búsqueda, la cantidad de Tweets recopilados hasta el momento, el tamaño del archivo, como también la datos de la duracióon de la prueba, entre otros datos que nos parecieron importantes.

In [ ]:
import os
import sys
import time
from datetime import datetime
from TwitterAPI import (
    TwitterAPI,
    TwitterOAuth,
    TwitterRequestError,
    TwitterConnectionError,
    HydrateType,
    OAuthType,
)
import json

def stream_tweets(query, expansions, tweet_fields, user_fields):

    datetimestart = datetime
    try:
        o = TwitterOAuth.read_file("credentials.txt")
        api = TwitterAPI(
            o.consumer_key,
            o.consumer_secret,
            auth_type=OAuthType.OAUTH2,
            api_version="2",
        )

        # DELETE STREAM RULES
        r = api.request("tweets/search/stream/rules", method_override="GET")
        rules = r.json()
        if "data" in rules:
            ids = list(map(lambda rule: rule["id"], rules["data"]))
            api.request("tweets/search/stream/rules", {"delete": {"ids": ids}})

        # ADD STREAM RULES
        r = api.request("tweets/search/stream/rules", {"add": [{"value": query}]})
        print(f"[{r.status_code}] RULE ADDED: {json.dumps(r.json(), indent=2)}\n")
        if r.status_code != 201:
            exit()

        # GET STREAM RULES

        r = api.request("tweets/search/stream/rules", method_override="GET")
        print(f"[{r.status_code}] RULES: {json.dumps(r.json(), indent=2)}\n")
        if r.status_code != 200:
            exit()

        # START STREAM

        r = api.request(
            "tweets/search/stream",
            {
                "expansions": expansions,
                "tweet.fields": tweet_fields,
                "user.fields": user_fields,
            },
            hydrate_type=HydrateType.APPEND,
        )

        if r.status_code != 200:
            exit()
        
        if not os.path.exists("data.json"):
            open("data.json", "x", encoding="utf-8")

        with open("data.json", "r+", encoding="utf-8") as file:
            cantidad_tweets = len(file.readlines())
            file.seek(0, os.SEEK_END)

            print("------------------------------------------------------------")
            datetimestart = datetime.now()
            print(
                (
                    "Proceso de recopilación iniciado: "
                    + datetimestart.strftime("%d/%m/%Y %H:%M:%S")
                )
            )

            for item in r:
                json.dump(item, file, ensure_ascii=False, indent=None)
                file.write("\n")

                cantidad_tweets += 1
                sys.stdout.write(
                    f"\rTamaño actual del archivo: {file.tell() / 1000} kb | Cantidad de tweets: {cantidad_tweets}"
                )

    except KeyboardInterrupt:
        datetimeend = datetime.now()
        print("\nProceso terminado: " + datetimeend.strftime("%d/%m/%Y %H:%M:%S"))
        datetimeend = datetimeend - datetimestart
        print("Duración de la prueba " + (str(datetimeend)) + " horas/minutos/segundos")
        print("------------------------------------------------------------")

    except TwitterRequestError as e:
        print(f"\n{e.status_code}")
        for msg in iter(e):
            print(msg)
    except TwitterConnectionError as e:
        print(e)
        print("tce")
    except Exception as e:
        print(e)
        print("e")


QUERY = "cambio climático OR sequías OR calentamiento global OR economía circular OR espacios verde OR protección ambiental lang:es -is:retweet"
EXPANSIONS = "author_id,referenced_tweets.id,referenced_tweets.id.author_id,in_reply_to_user_id,attachments.media_keys,attachments.poll_ids,geo.place_id,entities.mentions.username"
TWEET_FIELDS = "author_id,conversation_id,created_at,entities,geo,id,lang,public_metrics,source,text"
USER_FIELDS = "created_at,description,entities,location,name,profile_image_url,public_metrics,url,username"

stream_tweets(QUERY, EXPANSIONS, TWEET_FIELDS, USER_FIELDS)

***

## Consulta por fechas y horas

En ésta sección decidimos utilizar la estructura de un ***INDICE INVERTIDO***. Elegimos utilizar como ***KEY*** el usuario creador de cada tweet y, el número de línea donde se encuentra el tweet, como su correspondiente conjunto.

Cuando se realiza una consulta, si se propociona el usuario, se realiza una búsquerda dentro del mismo. De todos los tweets obtenidos, filtramos aquellos que concuerden con aquellas fechas proporcionadas por la consulta.

En el caso que no se proporcione ninguna fecha, utilizamos un rango mínimo-máximo de esta misma para poder continuar con la búsqueda.

En el caso que no se proporcione un usuario, se realiza una búsqueda lineal debido a la imposibilidad de hacerlo con una ***BINARY SEARCH***

(La siguiente imagen es a modo de ilustración y breve explicación del funcionamiento de un ***INDICE INVERTIDO*** )

![alt text](inverted_index.png "Indice Invertido")

In [ ]:
import datetime

from nltk.stem import SnowballStemmer  # Stemmer
from nltk.corpus import stopwords  # Stopwords
import json
import os
import string
import time
import re

class CreacionDeBloques:
    def __init__(self, documento, salida, temp="./temp", language='spanish'):
        ''' documentos: carpeta con archivos a indexar
            salida: carpeta donde se guardará el índice invertido'''
        self.documento = documento
        self.salida = salida
        self._blocksize = 5000
        self._temp = temp
        self._stop_words = frozenset(stopwords.words(language))  # lista de stop words
        self._stemmer = SnowballStemmer(language, ignore_stopwords=False)
        self._term_to_termID = {}
        self._user_to_userID = {}

        self.__indexar()

    def __lematizar(self, palabra):
        ''' Usa el stemmer para lematizar o recortar la palabra, previamente elimina todos
        los signos de puntuación que pueden aparecer. El stemmer utilizado también se
        encarga de eliminar acentos y pasar todo a minúscula, sino habría que hacerlo
        a mano'''

        # palabra = palabra.decode("utf-8", ignore).encode("utf-8")
        palabra = palabra.strip(string.punctuation + "»" + "\x97" + "¿" + "¡" + "\u201c" + \
                                "\u201d" + "\u2014" + "\u2014l" + "\u00bf")
        # "\x97" representa un guión

        palabra_lematizada = self._stemmer.stem(palabra)
        return palabra_lematizada

    def __indexar(self):
        n = 0
        lista_bloques_palabras = []
        lista_bloques_usuarios= []
        for bloque_palabras,bloque_usuarios in self.__parse_next_block():
            bloque_invertido_palabras = self.__invertir_bloque(
                bloque_palabras)            # ahora cada bloque tine cada palabra con todos los tweets de esa palabra

            bloque_invertido_usuarios = self.__invertir_bloque(
                bloque_usuarios)            # ahora cada bloque tine cada usuario con todos los tweets de esa usuario

            lista_bloques_palabras.append(self.__guardar_bloque_intermedio(bloque_invertido_palabras, n))
            lista_bloques_usuarios.append(self.__guardar_bloque_intermedio(bloque_invertido_usuarios, f"u{n}"))
            n += 1
        start = time.process_time()
        self.__intercalar_bloques(lista_bloques_palabras, self._term_to_termID, "postings")
        self.__intercalar_bloques(lista_bloques_usuarios,self._user_to_userID,"user_postings")
        end = time.process_time()
        print("Intercalar Bloques Elapsed time: ", end - start)

        self.__guardar_diccionario_terminos(self._term_to_termID, "terminos")
        self.__guardar_diccionario_terminos(self._user_to_userID, "usuarios")

    def __invertir_bloque(self, bloque):
        bloque_invertido = {}
        bloque_ordenado = sorted(bloque, key=lambda tupla: (tupla[0], tupla[1]))
        for par in bloque_ordenado:
            posting = bloque_invertido.setdefault(par[0], set())
            posting.add(par[1])
        return bloque_invertido

    def __guardar_bloque_intermedio(self, bloque, nro_bloque):
        archivo_salida = "b" + str(nro_bloque) + ".json"
        archivo_salida = os.path.join(self._temp, archivo_salida)
        for clave in bloque:
            bloque[clave] = list(bloque[clave])
        with open(archivo_salida, "w+") as contenedor:
            json.dump(bloque, contenedor)
        return archivo_salida

    def __intercalar_bloques(self, temp_files, term_to_termID, nombre_archivo_salida):

        lista_termID = [str(value) for value in term_to_termID.values()]
        iter_lista = iter(lista_termID)
        cantidad_term_group = len(lista_termID) // 1000 + 1

        posting_file = os.path.join(self.salida, f"{nombre_archivo_salida}.json")
        open_files = [open(f, "r") for f in temp_files]

        with open(posting_file, "w+") as salida:

            for x in range(cantidad_term_group):
                postings = {}
                lista_parte_term_ID = [next(iter_lista, None) for i in range(1000)]

                for data in open_files:
                    data.seek(0)
                    bloque = json.load(data)

                    i = 0
                    while i < 1000 and lista_parte_term_ID[i]:
                        try:
                            postings[i] = postings.setdefault(i,set()).union(set(bloque[lista_parte_term_ID[i]]))
                        except:
                            pass
                        i += 1
                for posting in postings.values():
                    json.dump(list(posting), salida, indent=None)
                    salida.write('\n')

    def __guardar_diccionario_terminos(self, term_to_termID, nombre_archivo_diccionario):
        path = os.path.join(self.salida, f"diccionario_{nombre_archivo_diccionario}.json")
        with open(path, "w") as contenedor:
            for term, termID in term_to_termID.items():
                json.dump((term, termID), contenedor)
                contenedor.write("\n")

    def __parse_next_block(self):
        n = self._blocksize  # espacio libre en el bloque actual
        termID = 1  # inicializamos el diccionario de términos
        userID = 1  # inicializamos el diccionario de términos
        bloque_palabras = []  # lista de pares (termID, tweetID)
        bloque_usuarios = []

        tweetID = 1  # ID de cada tweet, se puede acceder directament desde el json
        with open(self.documento, encoding="utf-8") as file:
            for tweet in file:
                n -= 1
                #Recorro las palabras
                palabras = json.loads(tweet)['data']['text'].split()  # va palabra por palabra del tweet
                for pal in palabras:
                    if pal not in self._stop_words:
                        pal = self.__lematizar(pal)
                        if pal not in self._term_to_termID:
                            self._term_to_termID[pal] = termID
                            termID += 1
                        bloque_palabras.append((self._term_to_termID[pal], tweetID))

                #Recorro los usuarios
                usuario = json.loads(tweet)["data"]["author_id_hydrate"]["username"]
                if usuario not in self._user_to_userID:
                    self._user_to_userID[usuario] = userID
                    userID += 1
                bloque_usuarios.append((self._user_to_userID[usuario], tweetID))

                tweetID += 1
                if n <= 0:
                    yield(bloque_palabras,bloque_usuarios)
                    n = self._blocksize
                    bloque_palabras = []
                    bloque_usuarios = []
            yield(bloque_palabras,bloque_usuarios)

***

## Consulta por palabras

Al seleccionar la opción de consultar por palabras solicitamos la cantidad de tweets (m número de los mismos) y la consulta a realizar. Manejamos que el número de tweets sea mayor que cero, de lo contrario, lanzamos una ValueError Exception, con el mensaje de que la cantidad de tweets no es válida.

A la hora de realizar la consulta analizamos aquellos operadores proporcionados. Estos mismos son válidos si son ***OR - AND - NOT*** y también que la query esté completa. Si la misma no tiene un buen formato, levantamos una Exception propia (***BadQueryFormat***)

Si todas las validaciones previas son correctas, se ejecuta la consulta solicitada.

In [ ]:
def _buscar_palabra(palabra):
    if (palabra != "*"):
        palabra = _lematizar(palabra.strip('"'))
        with open("./output/diccionario_terminos.json", "r") as contenedor:
            linea = next(contenedor, False)
            while (linea):
                linea = json.loads(linea)
                if (linea[0] == palabra):
                    break
                else:
                    linea = next(contenedor, False)
    else: linea = "Comodin"

    conjunto = set()
    if (linea == "Comodin"):
        with open("./output/postings.json", "r") as contenedor:
            while(contenedor):
                try: conjunto.update(json.loads(next(contenedor)))
                except StopIteration: break
    elif (linea):
        with open("./output/postings.json", "r") as contenedor:
            for i in range(1, linea[1]):
                valor = next(contenedor)
            conjunto.update(json.loads(next(contenedor)))

    return conjunto

def _buscar_palabras(query):
    matches = re.findall(r'\([^()]+\)|\"(?:[^\"]+)\"|and not|and|not|or', query)

    if (len(matches) % 2 == 0): 
        if (matches[0] == "not"):
            matches[0] = "and not"
            matches.insert(0, "*")
        else: 
            raise BadQueryFormat("Falta un operador que vincule dos términos: " + query)

    out = set()
    for i in range(0, len(matches), 2):
        if matches[i][0] == "(":
            conjunto = _buscar_palabras(matches[i].strip("()"))
        elif (type(matches[i]) != type(set)):
            texto = re.findall(r'\S+', matches[i])

            conjunto = set()
            if (len(texto) > 1):
                for palabra in texto:
                    conjunto.update(_buscar_palabra(palabra))
            else:
                conjunto = _buscar_palabra(matches[i])

        operador = ""
        if ((i - 1) > 0):
            operador = matches[i - 1]
        elif (i == 0):
            out.update(conjunto)

        if (operador == "and"):
            out.intersection_update(conjunto)
        elif (operador == "or"):
            out.update(conjunto)
        elif (operador == "and not" or operador == "not"):
            out.difference_update(conjunto)

    return out

***

## Agrupación de Tweets

En éste caso requerimos los nombres de:

1. rimer archivo
2. Segundo archivo
3. Archivo resultante

(Estos mismos deben estar con la extensión ***.json***)

Manejamos que los nombres proporcionados cumplan con los requerimientos solicitados previamente, caso contrario, levantamos una ValueError Exception.

Si todas las comprobaciones cumplen con lo esperado, realizamos la agrupación/unión correspondiente de los archivos.

Este proceso comienza con la búsqueda y comprobación previa de los archivos presentados previamente. Realizamos una obtención de tweets y sus correspondientes fechas. AL tener toda la información corespondiente, analizamos que cada sección cumpla con la existencia de una fecha y comparaciones entre las mismas para la correcta inserción de la información en el nuevo archivo.

In [ ]:
def agrupar_tweets_ordenados(file1, file2, salida):

    if not(os.path.exists(file1) and os.path.exists(file2)):
        print("Los archivos no existen")
        return
    with open(file1, "r", encoding="utf-8") as archivo_tweets1, open(file2, "r", encoding="utf-8") as archivo_tweets2, open(salida, "w", encoding="utf-8") as conjunto_tweets:

        tweet_archivo1, fecha_tweet1 = _obtener_tweet_y_fecha(archivo_tweets1)
        tweet_archivo2, fecha_tweet2 = _obtener_tweet_y_fecha(archivo_tweets2)

        while(tweet_archivo1 or tweet_archivo2):
                if(not fecha_tweet1):
                    json.dump(tweet_archivo2, conjunto_tweets,ensure_ascii=False, indent=None)
                    tweet_archivo2, fecha_tweet2 = _obtener_tweet_y_fecha(archivo_tweets2)

                elif(not fecha_tweet2):
                    json.dump(tweet_archivo1, conjunto_tweets,ensure_ascii=False, indent=None)
                    tweet_archivo1, fecha_tweet1 = _obtener_tweet_y_fecha(archivo_tweets1)

                elif(fecha_tweet1 < fecha_tweet2):
                    json.dump(tweet_archivo1, conjunto_tweets,ensure_ascii=False, indent=None)
                    tweet_archivo1, fecha_tweet1 = _obtener_tweet_y_fecha(archivo_tweets1)

                else:
                    json.dump(tweet_archivo2, conjunto_tweets,ensure_ascii=False, indent=None)
                    tweet_archivo2, fecha_tweet2 = _obtener_tweet_y_fecha(archivo_tweets2)
                conjunto_tweets.write("\n")

def _obtener_tweet_y_fecha(archivo_tweet):
    tweet = next(archivo_tweet, None)
    fecha = None
    while(tweet and not fecha):
        try:
            tweet = json.loads(tweet)
            fecha = datetime.strptime(tweet["data"]["created_at"],
                                     '%Y-%m-%dT%H:%M:%S.%fZ')
        except KeyError:
            print("KeyError")
            tweet = next(archivo_tweet, None)
    return(tweet, fecha)

***